# Exercise 1

In [ ]:
import pandas as pd
import csv
from bs4 import BeautifulSoup
import requests
import time
from collections import defaultdict
import numpy as np
import json
import pandas as pd
from IPython import get_ipython
from scipy import spatial
import heapq
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

## 1.1. Get the list of movies

In [ ]:
# The follow code take all the link which are in files movies 1, 2 ,3 and put them in list "link"

In [ ]:
links = []
movies1=open(r'C:\Users\simo2\OneDrive\Desktop\arishomework3\filmproject.html', "r")
soup_movies1 = BeautifulSoup(movies1, 'lxml')

movies2=open(r'C:\Users\simo2\OneDrive\Desktop\arishomework3\filmproject2.html', "r")
soup_movies2 = BeautifulSoup(movies2, 'lxml')

movies3=open(r'C:\Users\simo2\OneDrive\Desktop\arishomework3\filmproject3.html', "r")
soup_movies3 = BeautifulSoup(movies3, 'lxml')

for tag in soup_movies1.find_all('a', href=True):
    links.append(tag['href'])

for tag in soup_movies2.find_all('a', href=True):
    links.append(tag['href'])
    
for tag in soup_movies3.find_all('a', href=True):
    links.append(tag['href'])
print(links)    




## 1.2. Crawl Wikipedia

In [ ]:
# At this point we have a for loop through the list "links" and we  obtain the different lists of urls

In [ ]:
i = 0
for l in links:
    try:
        response = requests.get(l)
        soup = BeautifulSoup(response.text, 'html.parser')
        time.sleep(2)
        path_directory=r'C:\Users\HP\Desktop\file hw3\article_'+ str(i) +'.html'
        
        with open(path_directory, "w", encoding='utf-8') as file:
            file.write(str(soup))
            file.close()
        i+=1    
    except:
        time.sleep(1210) 
        response = requests.get(l)
        soup = BeautifulSoup(response.text, 'html.parser')
        time.sleep(2)
        path_directory=r'C:\Users\HP\Desktop\file hw3\article_'+ str(i) +'.html'
        
        with open(path_directory, "w", encoding='utf-8') as file:
            file.write(str(soup))
            file.close()
        i+=1


## 1.3 Parse downloaded pages

In [ ]:
#For each link we take title, and the first 2 sections and write a tsv file for each wiki page

In [ ]:
time = 0
while time <=30000:
    try:
        with open(r'C:\Users\HP\Desktop\file hw3\article_'+ str(time) +'.html', "r", encoding='utf-8') as file:
            articolo=file.read()
        soup = BeautifulSoup(articolo,'html.parser')
        
        
        title=soup.find_all('title')        
        title = title[0].get_text()
        title = title.split('-')
        title = str(title[0])
        mydiv = soup.find_all("div", class_="mw-parser-output")
        mydiv=mydiv[0] 
        par_and_head=mydiv.find_all(["p", "h2","h3","h4","h5","h6"])
        headings_list = ['h2','h3','h4','h5','h6']
        intro = ''
        plot = ''
        for i in range(len(par_and_head)):
            if par_and_head[i].name=='p':
                intro += par_and_head[i].get_text()
            if par_and_head[i].name in headings_list and intro != '':
                variabile = i
                break   
        while variabile < len(par_and_head):
            if par_and_head[variabile].name == 'p':
                plot += par_and_head[variabile].get_text()
            elif par_and_head[variabile].name in headings_list and plot != '':
                break
            variabile+=1
        
           
        table=soup.find('table',{'class':'infobox vevent'})
        article_table_dataframe = pd.read_html(str(table))
        article_table_dataframe=article_table_dataframe[0]
        article_table_dataframe.columns = ['Category','info']

        for i in range(len(article_table_dataframe)):
            for j in range(len(article_table_dataframe['info'][i])):
                if j!= 0:
                    if article_table_dataframe['info'][i][j].isupper() and article_table_dataframe['info'][i][j-1].islower():
                       article_table_dataframe['info'][i]= article_table_dataframe['info'][i][:j]+' '+article_table_dataframe['info'][i][j:]
        data_category = ['Title','Intro','Plot']
        data_to_merge1 = pd.DataFrame(data_category)
        data_tip = [title,intro,plot]
        data_to_merge2 = pd.DataFrame(data_tip)
        data_tip_merged=pd.merge(data_to_merge1, data_to_merge2, left_index=True, right_index=True) 
        data_tip_merged.columns = ['Category', 'info']
        film_info = pd.concat([article_table_dataframe, data_tip_merged], ignore_index=True)
        film_info=film_info.dropna(how='all')
        film_info=film_info.reset_index(drop=True)
        data_info_names=['Title','Intro','Plot', 'Directed by', 'Produced by', 'Written By', 'Starring', 'Music by', 'Release date', 'Running time', 'Country', 'Language', 'Budget']
        dataframe_info_names=pd.DataFrame(data_info_names)
        dataframe_info_names.columns=['Category']
        final_film_dataframe=pd.merge(dataframe_info_names, film_info, on='Category',how='left')
        with open(r'C:\Users\HP\Desktop\file hw3\wiki\article_'+ str(time)+'.tsv', 'wt',encoding='utf-8') as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
           
            tsv_writer.writerow(final_film_dataframe['Category'])   
            tsv_writer.writerow(final_film_dataframe['info'])
        time+=1
    except:
        
        
        with open(r'C:\Users\HP\Desktop\file hw3\article_'+ str(time) +'.html', "r", encoding='utf-8') as file:
            articolo=file.read()
        soup = BeautifulSoup(articolo,'html.parser')
        title=soup.find_all('title')        
        
        title = title[0].get_text()
        title = title.split('-')
        title = str(title[0])
        mydiv = soup.find_all("div", class_="mw-parser-output")
        if len(mydiv)!=0:
            mydiv=mydiv[0] 
            par_and_head=mydiv.find_all(["p", "h2","h3","h4","h5","h6"])
            headings_list = ['h2','h3','h4','h5','h6']
            intro = ''
            plot = ''
            for i in range(len(par_and_head)):
                if par_and_head[i].name=='p':
                    intro += par_and_head[i].get_text()
                if par_and_head[i].name in headings_list and intro != '':
                    variabile = i
                    break   
            while variabile < len(par_and_head):
                if par_and_head[variabile].name == 'p':
                    plot += par_and_head[variabile].get_text()
                elif par_and_head[variabile].name in headings_list and plot != '':
                    break
                variabile+=1
            data_category = ['Title','Intro','Plot']
            data_to_merge1 = pd.DataFrame(data_category)
            data_tip = [title,intro,plot]
            data_to_merge2 = pd.DataFrame(data_tip)
            data_tip_merged=pd.merge(data_to_merge1, data_to_merge2, left_index=True, right_index=True) 
            data_tip_merged.columns = ['Category', 'info']
            data_tip_merged=data_tip_merged.dropna(how='all')
            data_tip_merged=data_tip_merged.reset_index(drop=True)
            with open(r'C:\Users\HP\Desktop\file hw3\wiki\article_'+str(time)+'.tsv', 'wt',encoding='utf-8') as out_file:
                tsv_writer = csv.writer(out_file, delimiter='\t')
                tsv_writer.writerow(data_tip_merged['Category'])
                #aggiungere row con informazioni generiche   
                tsv_writer.writerow(data_tip_merged['info'])
            time+=1
        else:
            data_category = ['Title','Intro','Plot']
            data_to_merge1 = pd.DataFrame(data_category)
            data_tip = [title]
            data_to_merge2 = pd.DataFrame(data_tip)
            data_tip_merged=pd.merge(data_to_merge1, data_to_merge2, left_index=True, right_index=True) 
            data_tip_merged.columns = ['Category', 'info']
            data_tip_merged=data_tip_merged.dropna(how='all')
            data_tip_merged=data_tip_merged.reset_index(drop=True)
            with open(r'C:\Users\HP\Desktop\file hw3\wiki\article_'+str(time)+'.tsv', 'wt',encoding='utf-8') as out_file:
                
                tsv_writer = csv.writer(out_file, delimiter='\t')
               
                tsv_writer.writerow(data_tip_merged['Category'])   
                tsv_writer.writerow(data_tip_merged['info'])
            time+=1

# Exercise 2 Search Engine

In [ ]:
# As a first common step, we preprocess the documents by removing stopwords,removing punctuation and stemming
#We do this with a function called stringcleaning()
def string_cleaning(text):
    if type(text) is str:
        
        tokenizer = RegexpTokenizer(r'\w+')
        words = tokenizer.tokenize(text)
        
        stopWords = set(stopwords.words('english'))
        wordsFiltered = []
        for w in words:
            if w not in stopWords:
                wordsFiltered.append(w)
        
        porter = PorterStemmer()
        listOfWords = [porter.stem(word) for word in wordsFiltered]
        result = ' '.join(listOfWords)
    else:
        result = np.nan
    return result  


## Conjunctive query

In [ ]:
# We create a vocabulary in json format that maps each word to an integer (term_id)
def building_and_save_vocabulary(n,path):
    vocabulary = {}
    integer = 1
    for i in range(1, n):
        try:
            df = pd.read_csv(path+str(i)+'.tsv',encoding = 'utf-8', sep="\t")#Create a dataframe for each tsv
            intro_and_plot = df['Intro'][0].split()+df['Plot'][0].split()#Save as a list the Intro and Plot section of each tsv
            for word in intro_and_plot:
                        if word not in vocabulary:
                            vocabulary[word] = integer
                            integer += 1 
        except:
            continue                       
    json.dump(vocabulary, open('vocabulary.json', 'w', encoding='utf-8'))                        
    json.load(open('vocabulary.json', 'r')) 
    return vocabulary
vocabulary = building_and_save_vocabulary(30001,'C:/Users/simo2/tsv files project aris final and cleaned/articolo')
vocabulary

In [ ]:
#We create the Inverted index
#This function creates an Inverted Index dictionary,so each integer(that refers to a word) has as values the documents in which is contained
#We use the dictionary that we have created
def invertedIndexAdd(n,vocabulary,path):
    Inverted_index={}
    for i in range(1,n):
        try:
            df = pd.read_csv(path+str(i)+'.tsv',encoding = 'utf-8', sep="\t")
            intro_and_plot = df['Intro'][0].split()+df['Plot'][0].split()
            if intro_and_plot!=[]:        
                for word in intro_and_plot:
                    term_id=vocabulary[word]
                    if term_id not in Inverted_index:
                        s = set()
                        a='document_'+str(i)
                        s.add(a)
                        Inverted_index[term_id]=s
                    else:
                        s2 = Inverted_index[term_id]
                        a = 'document_'+str(i)
                        s2.add(a)
                        Inverted_index[term_id]=s2
        except:
            continue               
    return Inverted_index
Inverted_index=invertedIndexAdd(30001,vocabulary,'C:/Users/simo2/tsv files project aris final and cleaned/articolo')
Inverted_index

## First query execution

In [28]:
query='Disney movies 2019'
cleaned_query=string_cleaning(query).split()#We clean the string
cleaned_query = [vocabulary[key] for key in cleaned_query  if key in vocabulary]#Integer from the vocabulary of the query's words 
#This lines of codes helps us to make a list of the documents that contain all of the query's words
documents_appearances = set(Inverted_index[cleaned_query[0]])
for i in range(1, len(cleaned_query)):
            documents_appearances = documents_appearances.intersection(Inverted_index[cleaned_query[i]])
documents_appearances=list(documents_appearances)
#We want to create a dataframe to display the output
conjunctive_film_dataframe=pd.DataFrame(columns=['Title','Intro','Wikipedia Url'])#We create an empty dataframe
for i in range(len(documents_appearances)):#We make a for loop:We retrieve the tsv,we clean it and we append it to the conjunctive film dataframe
    number_of_document=(documents_appearances[i].split('_'))[1]
    film_dataframe=pd.read_csv(r'C:\Users\simo2\tsv files project aris\articolo'+number_of_document+'.tsv',encoding = 'utf-8', sep="\t")
    film_dataframe=film_dataframe[['Title','Intro']]
    film_dataframe['Wikipedia Url']=links[int(number_of_document)-1]
    conjunctive_film_dataframe=conjunctive_film_dataframe.append(film_dataframe,ignore_index=True)
conjunctive_film_dataframe

,Title,Intro,Wikipedia Url
0,The Avengers (2012 film),\r\n\r\nMarvel's The Avengers[6] (classified u...,https://en.wikipedia.org/wiki/The_Avengers_(20...
1,Thumbelina (1994 film),Thumbelina (also known as Hans Christian Ander...,https://en.wikipedia.org/wiki/Thumbelina_(1994...
2,Frozen (2013 film),\r\n\r\nFrozen is a 2013 American 3D computer-...,https://en.wikipedia.org/wiki/Frozen_(2013_film)
3,The Chronicles of Narnia: Prince Caspian,\r\nThe Chronicles of Narnia: Prince Caspian i...,https://en.wikipedia.org/wiki/The_Chronicles_o...
4,Cars 3,\r\n\r\nCars 3 is a 2017 American 3D computer-...,https://en.wikipedia.org/wiki/Cars_3


## Conjunctive query & Ranking score

In [ ]:
#This function returns a dictonary that maps for each word the list of documents in which it is contained in, and the relative tfIdf score
def Index_TFIDF(n,vocabulary,Inverted_index,path):
    
    inv_ind_TFIDF = defaultdict(list) 
                    
    for i in range(1,n):
        try:
            df = pd.read_csv(path+str(i)+'.tsv',encoding = 'utf-8', sep="\t")
            
            intro_and_plot = df['Intro'][0].split()+df['Plot'][0].split()
            dict_TF = defaultdict(int)           
            for word in intro_and_plot:
                term_id = vocabulary[word]
                dict_TF[term_id] += 1
                
                        
                #TFIDF
            for term_id in dict_TF:
                inv_ind_TFIDF[term_id].append(('document_'+str(i), (dict_TF[term_id]/len(intro_and_plot)) * (np.log(20000/(len(Inverted_index[term_id]))))))
        except:
            continue
    np.save('inv_ind_TFIDF.npy', inv_ind_TFIDF)
    inv_ind_TFIDF = np.load('inv_ind_TFIDF.npy',allow_pickle=True).item()
    return inv_ind_TFIDF
index_tfidf=Index_TFIDF(30001,vocabulary,Inverted_index,'C:/Users/simo2/tsv files project aris final and cleaned/articolo')

## Second query execution

In [29]:
k = int(input('Choose the number of documents you want as output\n--> '))#The user can choices how much documents he wants to display in the output
cleaned_query=string_cleaning(query).split()
#We create a TF dictionary of all the words in the query
dict_TF = defaultdict(float)
len_q = len(cleaned_query)
for word in cleaned_query:
    term_id = vocabulary[word]
    dict_TF[term_id] += 1/len_q

int_query = [vocabulary[key] for key in cleaned_query  if key in vocabulary]#The same as we did in the first search engine
cleaned_query = dict_TF #Our cleaned_query becomes the dict that we have just created
documents_appearances = set(Inverted_index[int_query[0]])#We do the same as we did in the first search engine
for i in range(1, len(int_query)):
            documents_appearances = documents_appearances.intersection(Inverted_index[int_query[i]])
documents_appearances=list(documents_appearances)
result = documents_appearances 
dict_list_doc_words = defaultdict(list)
for word in cleaned_query:#The tfidf of each word related to the document
        for elem in index_tfidf[word]:

            if elem[0] in result:
                dict_list_doc_words[elem[0]].append(elem[1]) 
heap = []#We create an heap structure to display only the first k documents as output
list_query = list(cleaned_query.values())
def cosine_similarity(array1,array2):#cosine similarity function
    return(1 - spatial.distance.cosine(array1, array2))
for doc in dict_list_doc_words:
    heapq.heappush(heap, (cosine_similarity(list_query, dict_list_doc_words[doc]), doc))
heap_result = heapq.nlargest(k, heap)#We have a list of tuples.Each tuple is the document and the similarity of this document with the query
similarity_film_dataframe=pd.DataFrame(columns=['Title','Intro','Wikipedia Url','Similarity'])
for i in range(len(heap_result)):
    number_of_document=(heap_result[i][1].split('_'))[1]
    film_dataframe=pd.read_csv(r'C:\Users\simo2\tsv files project aris\articolo'+number_of_document+'.tsv',encoding = 'utf-8', sep="\t")
    film_dataframe=film_dataframe[['Title','Intro']]
    film_dataframe['Wikipedia Url']=links[int(number_of_document)-1]
    film_dataframe['Similarity']=round(heap_result[i][0],2)
    similarity_film_dataframe=similarity_film_dataframe.append(film_dataframe,ignore_index=True) 
similarity_film_dataframe   

Choose the number of documents you want as output
--> 5


,Title,Intro,Wikipedia Url,Similarity
0,Thumbelina (1994 film),Thumbelina (also known as Hans Christian Ander...,https://en.wikipedia.org/wiki/Thumbelina_(1994...,0.91
1,The Avengers (2012 film),\r\n\r\nMarvel's The Avengers[6] (classified u...,https://en.wikipedia.org/wiki/The_Avengers_(20...,0.91
2,Cars 3,\r\n\r\nCars 3 is a 2017 American 3D computer-...,https://en.wikipedia.org/wiki/Cars_3,0.87
3,Frozen (2013 film),\r\n\r\nFrozen is a 2013 American 3D computer-...,https://en.wikipedia.org/wiki/Frozen_(2013_film),0.81
4,The Chronicles of Narnia: Prince Caspian,\r\nThe Chronicles of Narnia: Prince Caspian i...,https://en.wikipedia.org/wiki/The_Chronicles_o...,0.77
